# EECS3401 - Group Project - Group 2 #
    
## CO2 emission prediction ##

### frame ### 
supervised: as the target value(CO2) emmision is provided as label
        
model-based: regression model is used to make the prediction

batch-learning: as of now, we have no plan to make this model to be trained with future incomming new data

### 1. Dataset acquisition ###

In [1]:
%pip install pandas
%pip install requests



Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import requests as rq
from io import StringIO

In [4]:
url = 'https://raw.githubusercontent.com/jewbe22/eecs3401-group2-groupProject/main/data/Fuel_Consumption_Ratings_2023.csv'
download = rq.get(url).content

co2_emission = pd.read_csv(StringIO(download.decode(errors='ignore')), sep=',', engine='python')

### 2. EDA ###

In [5]:
# todo

### 3. Data cleaning/Preprocessing ###

In [6]:
# todo

### 4. Model training ###

In [7]:
# todo

### 5. Performance analysis ###

In [8]:
# todo